In [2]:
import clickhouse_connect
import datetime
import os
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv

import sys
sys.path.append('/home/boris/Documents/Work/analytics/Clickhouse')
from clickhouse_client import ClickHouse_client
ch = ClickHouse_client()
pd.set_option('display.max_rows', 1000)

___
## Tags: #Tables

# Links:

[[rep_mobile_citizens_id_city_partner]]

In [4]:
query_text = """--sql
    CREATE TABLE db1.t_activated_citizen_id
    (
        `report_date` Date,
        `address_uuid` String,
        `activated_citizen_id` UInt64,
        `flat_uuid` UInt64
    )
    ENGINE = MergeTree()
    ORDER BY address_uuid
    """
ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

In [13]:
start_date = datetime.datetime.strptime('2025-09-04','%Y-%m-%d').date()
end_date = datetime.datetime.strptime('2025-09-09','%Y-%m-%d').date()

dates_pd = pd.DataFrame({
        'date': pd.date_range(start=start_date, end=end_date).strftime('%Y-%m-%d'),
        'date_key': pd.date_range(start=start_date, end=end_date).strftime('%/year=%-Y/month=%m/%d%')
        })
dates_pd = dates_pd.iloc[::-1].reset_index(drop=True)
for day_index in dates_pd.index:
    date = dates_pd.loc[day_index,['date']].values[0]
    query_text = f"""
        INSERT INTO db1.t_activated_citizen_id
        SELECT
            report_date,
            address_uuid,
            countDistinctIf(citizen_id, citizen_id != 0) AS activated_citizen_id,	
            countDistinctIf(flat_uuid, flat_uuid != '') AS flat_uuid
        FROM db1.rep_mobile_citizens_id_city_partner
        WHERE address_uuid !=''
            AND report_date = '{date}'
        GROUP BY report_date,
                address_uuid
    """
    ch.query_run(query_text)
    print(date)

2025-09-09
2025-09-08
2025-09-07
2025-09-06
2025-09-05
2025-09-04


In [11]:
query_text = """--sql
    ALTER TABLE db1.t_activated_citizen_id DELETE WHERE report_date = '2025-09-09'
    """

ch.query_run(query_text)

""


In [ ]:
query_text = """--sql
    CREATE MATERIALIZED VIEW db1.t_activated_citizen_id_mv
    REFRESH EVERY 1 DAY OFFSET 5 HOUR 35 MINUTE TO db1.t_activated_citizen_id AS 
    SELECT
        report_date,
        address_uuid,
        countDistinctIf(citizen_id, citizen_id != 0) AS activated_citizen_id,	
        countDistinctIf(flat_uuid, flat_uuid != '') AS flat_uuid
    FROM db1.rep_mobile_citizens_id_city_partner
    WHERE address_uuid != ''
    GROUP BY report_date, address_uuid
    """
ch.query_run(query_text)

In [12]:
query_text = """--sql
SELECT
    *
FROM db1.t_activated_citizen_id
ORDER BY report_date DESC
LIMIT 10
    """
ch.query_run(query_text)

,report_date,address_uuid,activated_citizen_id,flat_uuid
0,2025-09-03,01ebfb5a-d1fc-431d-b14e-5d34cd9a60fd,274,124
1,2025-09-03,01def67c-9585-4135-96d8-b9b23cba4b29,19,14
2,2025-09-03,01df6257-b52b-4037-9e74-1b2e25698a1b,55,27
3,2025-09-03,01e855fd-e154-43d5-846b-a0b9fbf1818a,33,19
4,2025-09-03,01e98368-fbbe-401d-ba98-9fd48fe61ea8,3,2
5,2025-09-03,01da4f6b-b8c7-45a3-a0b9-88be04312ed3,19,10
6,2025-09-03,01ea7574-90df-48af-acf5-22e904eea9a7,6,4
7,2025-09-03,01ebf2f7-f3f3-43b3-9999-8a398c92ea4c,21,18
8,2025-09-03,01ea399b-fc46-4e83-a575-a4f303a523d1,17,6
9,2025-09-03,01eb6c51-f018-44be-8eb1-5813187d460f,52,24


In [2]:
query_text = """
    DROP TABLE db1.t_activated_citizen_id
    """
ch.query_run(query_text)

read_rows,read_bytes,written_rows,written_bytes,total_rows_to_read,result_rows,result_bytes,elapsed_ns,query_id
i64,i64,i64,i64,i64,i64,i64,i64,str
0,0,0,0,0,0,0,18537416,"""1f844653-5c80-4ab7-83e5-d723b5…"


In [3]:
query_text = """
    DROP TABLE db1.t_activated_citizen_id_mv
    """
ch.query_run(query_text)

read_rows,read_bytes,written_rows,written_bytes,total_rows_to_read,result_rows,result_bytes,elapsed_ns,query_id
i64,i64,i64,i64,i64,i64,i64,i64,str
0,0,0,0,0,0,0,2523682,"""5346be78-67b1-4600-82e3-d62ba8…"
